In [1]:
import pandas as pd
import os
import regex as re
path = "corpus"
corp_files = os.listdir(path)
corp_files

['eco_rzepa.csv',
 'eco_gpc.csv',
 'eco_dorzeczy.csv',
 'eco_wyborcza.csv',
 'eco_newsweek.csv',
 'eco_wprost.csv',
 'eco_wpolityce.csv',
 'eco_polityka.csv']

In [2]:
data_list = []
cols = ["text", "id", 'date', 'source']

for file in corp_files:
    df = pd.read_csv(path+"/"+file, parse_dates=['date'])
    if 'id' not in df.columns:
        df['id'] = df.index
    df = df[cols]    
    data_list.append(df)
df = pd.concat(data_list)

In [3]:
word_regex = r'(?:[AaĄąBbCcĆćDdEeĘęFfGgHhIiJjKkLlŁłMmNnŃńOoÓóPpRrSsŚśTtUuWwYyZzŹźŻż\d]+\s+)'
regex_expression = word_regex + r'{1,5}' + "zmian[^ ]* klimat[^ ]*\s+" + word_regex + r'{1,5}'

"(?:[AaĄąBbCcĆćDdEeĘęFfGgHhIiJjKkLlŁłMmNnŃńOoÓóPpRrSsŚśTtUuWwYyZzŹźŻż\d\.\-]+\s+){3}zmian[^ ]* klimat[^ ]*\s+(?:[AaĄąBbCcĆćDdEeĘęFfGgHhIiJjKkLlŁłMmNnŃńOoÓóPpRrSsŚśTtUuWwYyZzŹźŻż\d\.\-]+\s+){3}"

In [4]:
df['text_processed'] = df['text'].apply(lambda x: re.sub(r'[\n\.\-]', ' ', x))
df['text_processed'] = df['text_processed'].apply(lambda x: re.sub(r'\s+', ' ', x))
df['text_processed'] = df['text_processed'].apply(lambda x: re.sub(r'[^\dAaĄąBbCcĆćDdEeĘęFfGgHhIiJjKkLlŁłMmNnŃńOoÓóPpRrSsŚśTtUuWwYyZzŹźŻż ]', '', x))
df['text_processed'] = df['text_processed'].str.lower()

In [5]:
df['raw_colocates'] = df['text_processed'].apply(lambda x: re.findall(regex_expression, x))

In [6]:
df['count'] = df['raw_colocates'].apply(lambda x: len(x))
print(df['count'].sum())

8069


In [7]:
df['real_count'] = df['text_processed'].apply(lambda x: len(re.findall("zmian[^ ]* klimat[^ ]*", x)))
print(df['real_count'].sum())

8223


In [8]:
df[df['real_count'] > 0].source.value_counts()

source
Wyborcza                    1992
Rzeczpospolita              1374
Gazeta Polska Codziennie     252
POLITYKA                     225
Newsweek                     113
wPolityce                    105
WPROST                        65
DoRzeczy                      47
Name: count, dtype: int64

In [9]:
df_exploded = df[df['count'] > 0].explode('raw_colocates')
df_exploded.reset_index(drop=True, inplace=True)
df_exploded.rename(columns={'raw_colocates': 'colocate', 'id': 'text_id'}, inplace=True)
df_exploded['id'] = df_exploded.index
df_exploded.drop(columns=['text', 'text_processed', 'count', 'real_count'], inplace=True)
df_exploded.head()

,text_id,date,source,colocate,id
0,43,2015-01-02,Rzeczpospolita,rolnictwa i leśnictwa do postępujących zmian k...,0
1,78,2015-01-02,Rzeczpospolita,środowiska przeciwdziałaniu i adaptacji do zmi...,1
2,411,2015-01-08,Rzeczpospolita,powiększania się bądź ograniczania ubóstwa zmi...,2
3,411,2015-01-08,Rzeczpospolita,2008 r ustawa o przeciwdziałaniu zmianom klima...,3
4,1154,2015-01-16,Rzeczpospolita,ogłosić encyklikę koncentrującą się na zmianac...,4


In [10]:
df_exploded.to_csv("colocates.csv", index=False)